In [30]:
!pip install -qU langchain-groq

    pytz (>=2011n)
         ~~~~~~~^


In [31]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq

# Load environment variables from .env file
load_dotenv()
# Retrieve the API key from the environment variable
groq_api_key = os.getenv("GROQ_API_KEY")

# Initialize the model with the API key from the environment variable
llm = ChatGroq(
    model="mixtral-8x7b-32768",
    temperature=0,
    groq_api_key=groq_api_key,  # Using the environment variable here
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

response = llm.invoke("The first person to enter space")
print(response.content)


The first person to enter space was Yuri Gagarin, a Russian astronaut. He was launched into space on April 12, 1961, aboard the Vostok 1 spacecraft, as part of the Soviet Union's space program. Gagarin orbited the Earth once and spent 108 minutes in space before safely returning to Earth. His historic flight marked a significant milestone in the space race between the United States and the Soviet Union.


CHROMADB Understanding 


In [32]:
import chromadb

client = chromadb.Client()
collection = client.create_collection(name="collection")

UniqueConstraintError: Collection collection already exists

In [ ]:
collection.add(
    documents=[
        "This is a document from Mumbai",
        "Here is a document from dubai"
    ],
    ids=['id1', 'id2']
)

In [ ]:
all_docs = collection.get()
all_docs

{'ids': ['id1', 'id2'],
 'embeddings': None,
 'metadatas': [None, None],
 'documents': ['This is a document from Mumbai',
  'Here is a document from dubai'],
 'uris': None,
 'data': None}

In [ ]:
results = collection.query(
    query_texts=['Query is about big apple'],
    n_results=2
)
results

{'ids': [['id1', 'id2']],
 'distances': [[1.635650634765625, 1.7806391716003418]],
 'metadatas': [[None, None]],
 'embeddings': None,
 'documents': [['This is a document from Mumbai',
   'Here is a document from dubai']],
 'uris': None,
 'data': None}

In [ ]:

collection.delete(ids=all_docs['ids'])
collection.get()

{'ids': [],
 'embeddings': None,
 'metadatas': [],
 'documents': [],
 'uris': None,
 'data': None}

In [ ]:
collection.add(
    documents=[
        "This document is about New York",
        "This document is about Delhi"
    ],
    ids=["id3", "id4"],
    metadatas=[
        {"url": "https://en.wikipedia.org/wiki/New_York_City"},
        {"url": "https://en.wikipedia.org/wiki/New_Delhi"}
    ]
)

In [ ]:

results = collection.query(
    query_texts=["Query is about Chhole Bhature"],
    n_results=2
)
results

{'ids': [['id4', 'id3']],
 'distances': [[1.5588479042053223, 1.8114912509918213]],
 'metadatas': [[{'url': 'https://en.wikipedia.org/wiki/New_Delhi'},
   {'url': 'https://en.wikipedia.org/wiki/New_York_City'}]],
 'embeddings': None,
 'documents': [['This document is about Delhi',
   'This document is about New York']],
 'uris': None,
 'data': None}

In [ ]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-33460")
page_data = loader.load().pop().page_content
print(page_data)



Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsFIND A STORENIKE JOURNALBECOME A MEMBERFEEDBACKPROMO CODESGET HELPGET HELPOrder StatusShipping and DeliveryReturnsPayment OptionsContact

In [ ]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [ ]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

[{'role': 'Contract Jobs',
  'experience': None,
  'skills': None,
  'description': 'Explore the latest contract opportunities at Nike.'},
 {'role': 'Join The Talent Community',
  'experience': None,
  'skills': None,
  'description': 'Stay connected to Nike by joining our Talent Community.'},
 {'role': 'Life @ Nike',
  'experience': None,
  'skills': None,
  'description': "Learn about what it's like to work at Nike including benefits, brands, teams, and more."},
 {'role': 'Administrative Support',
  'experience': None,
  'skills': None,
  'description': 'Explore Administrative Support opportunities at Nike.'},
 {'role': 'Advanced Innovation',
  'experience': None,
  'skills': None,
  'description': 'Explore Advanced Innovation opportunities at Nike.'},
 {'role': 'Air Manufacturing Innovation',
  'experience': None,
  'skills': None,
  'description': 'Explore Air Manufacturing Innovation opportunities at Nike.'},
 {'role': 'Aviation',
  'experience': None,
  'skills': None,
  'descrip

In [ ]:
prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [ ]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)

In [ ]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [ ]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [ ]:
job = json_res
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

TypeError: list indices must be integers or slices, not str

In [ ]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

NameError: name 'job' is not defined